# global dimensionality-reduction (GDR) algorithm 示例

In [1]:
import numpy as np
import pandas as pd
from scipy import linalg
from scipy import optimize 
import functools

import tensorly
from tensorly.decomposition import partial_tucker
from tensorly.decomposition import tucker
tensorly.set_backend('numpy')

Using mxnet backend.
Using numpy backend.


### 生成数据集

In [2]:
tensor_steam_length = 300

factors_tensor_list = []
for i in np.arange(tensor_steam_length):
    a = np.random.normal(size=[69], scale=0.5)
    b = np.random.normal(size=[16], scale=0.5)
    c = np.random.normal(size=[32], scale=0.5)
    x = np.zeros([1, 69, 16, 32])
    x[0,:,0,0] = a
    x[0,1,:,1] = b
    x[0,2,2,:] = c
    factors_tensor_list.append(x)
factors_tensor = np.concatenate(factors_tensor_list)

targets = np.random.normal(scale=0.01, size=[300,1])

### 3.3 $\sum$ 循环向量化

$U^i_k\in\mathbb{R}^{batch \times I_k \times J_k}$

$U^{iT}_k\in\mathbb{R}^{batch \times J_k \times I_k}$

$M^i_k = U^i_k U^{iT}_k\in \mathbb{R}^{batch \times I_k \times I_k}$

$M^i_{k(0)} \in \mathbb{R}^{batch \times I_kI_k}$

令 $N = batch$

- $D_{U_k} = \sum\limits ^N_{i=1}d_{i,i}U^{i}_kU^{iT}_k \\=\sum\limits ^N_{i=1}d_{i,i}M^i_k\\= mat \{(vec(diag(D)))^T M_{k(0)}\}$
- $W_{U_k} = \sum\limits^N_{i=1}\sum\limits ^N_{j=1}w_{i,j}U^i_kU^{jT}_k\\= \sum\limits^N_{i=1}(\sum\limits ^N_{j=i}w_{i,j})U^i_kU^{jT}_k\\= \sum\limits^N_{i=1}(\sum\limits ^N_{j=i}w_{i,j})U^i_kU^{jT}_k\\=\sum\limits^N_{i=1}w_iU^i_kU^{jT}_k\\=\sum\limits^N_{i=1}w_i M^i_{k}\\=mat\{w M_{k(0)} \}$

其中

- $w_{i,j} = \begin{cases}1,\ if\ \ i \le j \ and \ \|y_i -y_j \| \le 5\% \\0, \ otherwise \end{cases} $
- $\sum\limits ^N_{j=1}w_{i,j} = w_i= sum(w_{i,:}, axis=1)$

In [3]:
def get_weighting_of_geometric_structure (targets):
    W = (targets - targets.T) / targets.T # 广播
    W = np.abs(W) - 0.05 # 转换绝对值并判断相似度
    W[W>0.0]=0
    W[W<0.0]=1
    upper_traingular_matrix = np.eye(W.shape[0]).cumsum(1) # 上三角矩阵掩码
    return W * upper_traingular_matrix

def get_var_of_adjusting_geometric_structure(targets):
    W = get_weighting_of_geometric_structure(targets)
    return np.expand_dims(W.sum(0),axis=0)

D = get_var_of_adjusting_geometric_structure(targets)
W = get_weighting_of_geometric_structure(targets)

### 循环tucker分解
计划迁移PyTorch构建批次处理

In [4]:
factors_tensor = tensorly.tensor(factors_tensor)

core_list = []
mode_factors_list = []
for i in range(factors_tensor.shape[0]):
    print (i)
    core, mode_factors= tucker(factors_tensor[i])
    core = np.expand_dims(core, axis=0)
    core_list.append(core)
    mode_factors_list.append(mode_factors)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

##### 连接张量流

In [5]:
batch_length = tensor_steam_length
a_list = []
b_list = []
c_list = []
for i in range(batch_length):
    a = np.expand_dims(mode_factors_list[i][0], axis=0)
    b = np.expand_dims(mode_factors_list[i][1], axis=0)
    c = np.expand_dims(mode_factors_list[i][2], axis=0)
    a_list.append(a)
    b_list.append(b)
    c_list.append(c)
    
U1 = np.concatenate(a_list)
U2 = np.concatenate(b_list)
U3 = np.concatenate(c_list)

M1 = U1 * np.transpose(U1, axes=[0,2,1])
M2 = U2 * np.transpose(U2, axes=[0,2,1])
M3 = U3 * np.transpose(U3, axes=[0,2,1])

#### 暂时没有找到较好的批次处理tucker分解的方法，这里特例处理

In [6]:
D_U1_core = np.matmul(D, tensorly.base.unfold(M1, mode=0))
I_k = np.int(np.sqrt(D_U1_core.shape[1]))
D_U1 = tensorly.base.fold(D_U1_core, mode=0, shape=[I_k, I_k])

D_U2_core = np.matmul(D, tensorly.base.unfold(M2, mode=0))
I_k = np.int(np.sqrt(D_U2_core.shape[1]))
D_U2 = tensorly.base.fold(D_U2_core, mode=0, shape=[I_k, I_k])

D_U3_core = np.matmul(D, tensorly.base.unfold(M3, mode=0))
I_k = np.int(np.sqrt(D_U3_core.shape[1]))
D_U3 = tensorly.base.fold(D_U3_core, mode=0, shape=[I_k, I_k])

In [7]:
vec_W = np.expand_dims(W.sum(axis=0), axis=0)

W_U1_core = np.matmul(vec_W, tensorly.base.unfold(M1, mode=0))
I_k = np.int(np.sqrt(W_U1_core.shape[1]))
W_U1 = tensorly.base.fold(W_U1_core, mode=0, shape=[I_k, I_k])

W_U2_core = np.matmul(vec_W, tensorly.base.unfold(M2, mode=0))
I_k = np.int(np.sqrt(W_U2_core.shape[1]))
W_U2 = tensorly.base.fold(W_U2_core, mode=0, shape=[I_k, I_k])

W_U3_core = np.matmul(vec_W, tensorly.base.unfold(M3, mode=0))
I_k = np.int(np.sqrt(W_U3_core.shape[1]))
W_U3 = tensorly.base.fold(W_U3_core, mode=0, shape=[I_k, I_k])

### 二次规划优化估计修正矩阵$V_k$
添加约束条件 $trace(V^TD_UV)=1$ 获得目标函数的唯一优化结果，这是获取因子矩阵对应的修正矩阵的优化问题转化为：

- $min\ J(V) = trace(V^TD_UV-V^TW_UV)$
- $s.t. \ trace(V^TD_UV)=1$

In [8]:
def objective_function(V, D_U, W_U, J_K):
    newshape = [D_U.shape[0], J_K]
    V = np.reshape(V,newshape=newshape)
    left = np.matmul(np.matmul(V.T, D_U),V)
    right = np.matmul(np.matmul(V.T, W_U),V)
    return np.trace(left + right)

def constraints(V, D_U):
    newshape = [D_U.shape[0], J_K]
    V = np.reshape(V,newshape=newshape)
    left = np.matmul(np.matmul(V.T, D_U),V)
    return np.trace(left)- 1.0

#### $V_1$

In [9]:
objective_function_1 = functools.partial(
    objective_function, D_U = D_U1, W_U = W_U1, J_K = 10)

constraints_1 = functools.partial(constraints, D_U = D_U1)

cons_1 = ({'type':'ineq', 'fun':constraints_1})
initial_1 = np.random.normal(scale=0.1, size=D_U1.shape[0]*10).reshape(D_U1.shape[0],10)

V1 = optimize.minimize(objective_function_1, initial_1).x.reshape(D_U1.shape[0],10)

#### $V_2$

In [10]:
objective_function_2 = functools.partial(
    objective_function, D_U = D_U2, W_U = W_U2, J_K = 10)

constraints_2 = functools.partial(constraints, D_U = D_U2)

cons_2 = ({'type':'ineq', 'fun':constraints_2})
initial_2 = np.random.normal(scale=0.1, size=D_U2.shape[0]*10).reshape(D_U2.shape[0],10)

V2 = optimize.minimize(objective_function_2, initial_2).x.reshape(D_U2.shape[0],10)

#### $V_3$

In [11]:
objective_function_3 = functools.partial(
    objective_function, D_U = D_U3, W_U = W_U3, J_K = 10)

constraints_3 = functools.partial(constraints, D_U = D_U3)

cons_3 = ({'type':'ineq', 'fun':constraints_3})
initial_3 = np.random.normal(scale=0.1, size=D_U3.shape[0]*10).reshape(D_U3.shape[0],10)

V3 = optimize.minimize(objective_function_3, initial_3).x.reshape(D_U3.shape[0],10)

#### $\bar{\mathcal{X}_i} = \mathcal{C}_i \times_1 (V^T_1U^i_1) \times_2 (V^T_2U^i_2) \times_3 (V^T_3U^i_3)$

In [12]:
new_U1 =  np.matmul(V1.T, U1)
new_U2 =  np.matmul(V2.T, U2)
new_U3 =  np.matmul(V3.T, U3)

In [13]:
unfold_mode1 = tensorly.base.partial_unfold(factors_tensor, mode=0, skip_begin=1)
times_mode1 = np.matmul(new_U1, unfold_mode1)
times1_shape = (factors_tensor.shape[0] ,new_U1.shape[1], factors_tensor.shape[2], factors_tensor.shape[3])
times1 = tensorly.base.partial_fold(times_mode1, 0, times1_shape, skip_begin=1, skip_end=0)

unfold_mode2 = tensorly.base.partial_unfold(times1, mode=1, skip_begin=1)
times_mode2 = np.matmul(new_U2, unfold_mode2)
times2_shape = (factors_tensor.shape[0] ,new_U1.shape[1] ,new_U2.shape[1], factors_tensor.shape[3])
times2 = tensorly.base.partial_fold(times_mode2, 1, times2_shape, skip_begin=1, skip_end=0)

In [14]:
unfold_mode3 = tensorly.base.partial_unfold(times2, mode=2, skip_begin=1)
times_mode3 = np.matmul(new_U3, unfold_mode3)
times3_shape = (factors_tensor.shape[0] ,new_U1.shape[1], new_U2.shape[1], new_U3.shape[1])
times3 = tensorly.base.partial_fold(times_mode3, 2, times3_shape, skip_begin=1, skip_end=0)

In [15]:
new_factors_tensor = times3

#### 动态关系捕获和降维 $\mathcal{X}\in\mathbb{R}^{I_1 \times I_2 \times I_3} \to \bar{\mathcal{X}} \in \mathbb{R}^{J_1 \times J_2 \times J_3}$

In [16]:
factors_tensor.shape

(300, 69, 16, 32)

In [17]:
new_factors_tensor.shape

(300, 10, 10, 10)

In [18]:
new_factors_tensor

array([[[[  1.20088711e+18,  -9.39257236e+18,   4.17323386e+18, ...,
            1.22671463e+19,   2.83522318e+18,  -8.96128282e+18],
         [ -1.85584727e+18,   1.48577156e+19,  -6.74303253e+18, ...,
           -1.81974692e+19,  -4.55194623e+18,   1.27374179e+19],
         [ -3.03252593e+18,   2.51420172e+19,  -1.17587833e+19, ...,
           -2.78207473e+19,  -7.86770073e+18,   1.80138729e+19],
         ..., 
         [  3.18533307e+18,  -2.46754559e+19,   1.08643517e+19, ...,
            3.30711160e+19,   7.40156045e+18,  -2.45479422e+19],
         [  1.61892482e+18,  -1.39250973e+19,   6.70826945e+18, ...,
            1.37387725e+19,   4.45022985e+18,  -7.98855173e+18],
         [ -2.59008049e+18,   1.89475885e+19,  -7.87689904e+18, ...,
           -2.93666789e+19,  -5.46299794e+18,   2.35803904e+19]],

        [[ -4.72346653e+17,   2.48191702e+18,  -5.90022662e+17, ...,
           -7.57496975e+18,  -5.07422353e+17,   7.53730440e+18],
         [ -1.70434742e+18,   2.08464029e+19,